<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Generating-Word-Embeddings---Lab" data-toc-modified-id="Generating-Word-Embeddings---Lab-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generating Word Embeddings - Lab</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Getting-Started" data-toc-modified-id="Getting-Started-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Getting Started</a></span></li><li><span><a href="#Preparing-the-Data" data-toc-modified-id="Preparing-the-Data-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Preparing the Data</a></span></li><li><span><a href="#Training-the-Model" data-toc-modified-id="Training-the-Model-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Training the Model</a></span></li><li><span><a href="#Examining-Our-Word-Vectors" data-toc-modified-id="Examining-Our-Word-Vectors-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Examining Our Word Vectors</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Summary</a></span></li></ul></li></ul></div>

# Generating Word Embeddings - Lab

## Introduction

In this lab, we'll learn how to generate our own word embeddings by training our own Word2Vec model, and also by building embedding layers right into our Deep Neural Networks!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of word2vec such as vector size and window size

## Getting Started

In this lab, we'll start by creating our own word embeddings by making use of the Word2Vec Model. Then, we'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in our model. 

The easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which we can use to begin training immediately. For this lab, we'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.  In this lab, we'll learn how to train a Word2Vec model on the text data to generate word embeddings for them. In the next lab, we'll then use the vectors created by our Word2Vec model to effectively train a classifier to predict the category of news given the headline and description of each article. In this lab, we won't do any classification, although we will learn how to train a Word2Vec model and explore the relationships between different word vectors in our embedding!

Run the cell below to import everything we'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, we'll import the data. You'll find the data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to also include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what our data looks like. 

In [2]:
raw_df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
raw_df.head(3)

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...


## Preparing the Data

Since we're working with text data, we'll still need to do some basic preprocessing and tokenize our data. You'll notice from the sample of the data above that two different columns contain text data--`headline` and `short_description`. The more text data our Word2Vec model has, the better it will perform. Therefore, we'll want to combine the two columns before tokenizing each comment and training our Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [6]:
raw_df['combined_text'] = raw_df.headline+' '+raw_df.short_description
data = raw_df['combined_text'].map(word_tokenize)

Let's inspect the first 5 items in `data` to see how everything looks. 

In [7]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

You'll notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Recall from our previous lesson on how Word2Vec works that we can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If our window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example we saw).

Now that we've prepared our data, let's train our model and explore a bit!

## Training the Model

We'll start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [8]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that we've created our Word2Vec model, we still need to train it on our model. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which we can find in `model.corpus_count`. 
    * The number of `epochs` we want to train for, which we'll set to `10`

In [10]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55565633, 67352790)

Great! We now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which we'll find stored inside of `model.wv`. For simplicity's sake, let's go ahead and store this inside of the variable `wv` in order to save ourselves some keystrokes down the line. 

In [11]:
wv = model.wv

## Examining Our Word Vectors

Now that we have a trained Word2Vec model, let's go ahead and explore the relationships between some of the words in our corpus! 

One cool thing we can use Word2Vec for is to get the most similar words to a given word. We can do this passing in the word to `wv.most_similar()`. 

In the cell below, let's try getting the most similar word to `'Texas'`.

In [12]:
wv.most_similar('Texas')

[('Louisiana', 0.8186634182929993),
 ('Maryland', 0.8045839071273804),
 ('Pennsylvania', 0.8007960319519043),
 ('Connecticut', 0.8005355596542358),
 ('Ohio', 0.794875979423523),
 ('Georgia', 0.787005603313446),
 ('Utah', 0.7868191003799438),
 ('Massachusetts', 0.7847108244895935),
 ('Illinois', 0.7791999578475952),
 ('Oklahoma', 0.7762464284896851)]

In [15]:
wv.most_similar('Smith')

[('Ferrell', 0.7079148888587952),
 ('Arnett', 0.6774768829345703),
 ('Wilson', 0.6721471548080444),
 ('Packer', 0.6715589761734009),
 ('Ward', 0.6526132225990295),
 ('Jada', 0.6478809714317322),
 ('Elliot', 0.6454682946205139),
 ('Dave', 0.6430126428604126),
 ('Joel', 0.6389086842536926),
 ('Bennett', 0.6387008428573608)]

Interesting! All of the most similar words are also states. 

We can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [16]:
wv.most_similar(negative='Texas')

[('Parent/Grandparent', 0.42077454924583435),
 ('Rank-and-file', 0.3859512209892273),
 ('Habitable', 0.3797561824321747),
 ('much-vaunted', 0.3754611611366272),
 ('Telecommunication', 0.3658772110939026),
 ('Trumpist', 0.36471349000930786),
 ('Strategist™', 0.36060330271720886),
 ('Likened', 0.35991257429122925),
 ('Babymetal', 0.35920125246047974),
 ('Animal-rescue', 0.35790348052978516)]

These seem like just noise. This is because of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with our given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

We can also get the vector for a given word by passing in the word as if we were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [17]:
wv['Texas']

array([-1.81224215e+00,  1.37538421e+00,  8.26056898e-01,  2.13949323e+00,
        6.84556007e-01, -2.30663761e-01, -5.20035565e-01,  4.67003882e-01,
        1.21330690e+00,  6.96900547e-01, -1.02232145e-02,  3.46440464e-01,
       -1.29261151e-01, -8.06416988e-01, -1.19861519e+00,  2.82659888e-01,
       -2.05618644e+00, -3.81869048e-01, -2.36682266e-01,  1.17538714e+00,
        2.10424709e+00, -5.43942511e-01,  4.94890034e-01,  5.88013649e-01,
       -3.42418838e+00,  6.62216604e-01, -1.38523901e+00,  9.28737581e-01,
       -8.94726634e-01, -1.13608909e+00,  1.67487192e+00, -1.14690617e-01,
       -1.85376823e-01,  1.11816800e+00, -5.78211069e-01, -1.55389178e+00,
       -1.11430860e+00,  2.66399860e+00, -1.00320792e+00, -1.25659847e+00,
        1.28438318e+00, -1.12907243e+00, -7.09942400e-01,  2.41539407e+00,
       -3.85202289e-01,  2.33913079e-01,  1.44872785e-01, -1.01183176e+00,
       -6.11137986e-01,  1.60941410e+00, -3.36066425e-01,  1.65326223e-01,
        8.36079180e-01,  

Let's get all of the word vectors from the object at once. We can find these inside of `wv.vectors`.  Do this now in the cell below.  

In [18]:
wv.vectors

array([[-2.70938396e-01, -1.29837215e+00,  5.43788075e-01, ...,
        -3.67638290e-01,  1.22068048e+00, -9.34507906e-01],
       [-5.34300029e-01, -8.76404345e-01, -1.03353369e+00, ...,
         7.67283618e-01,  5.38057610e-02, -3.30461645e+00],
       [-1.57401109e+00,  1.51680362e+00,  2.09100857e-01, ...,
         3.22135121e-01, -1.70227361e+00, -1.33297360e+00],
       ...,
       [ 1.32550821e-01, -1.09418202e-02,  8.83145556e-02, ...,
        -9.67986956e-02, -2.51237191e-02,  5.33956736e-02],
       [-2.45648772e-02,  1.23127745e-02, -1.13977375e-03, ...,
        -1.20750135e-02,  2.24978738e-02,  3.98584343e-02],
       [ 5.52824885e-02, -2.08601654e-02,  8.44334289e-02, ...,
        -3.75045054e-02,  1.86786875e-02,  9.95563902e-03]], dtype=float32)

As a final exercise, let's try recreating the _'king' - 'man' + 'woman' = 'queen'_ example we've seen before. We can do this by using the `most_similar` function and putting the things we want added together inside of an array passed to the `positive` parameter, and the things we want subtracted as an array passed to the the `negative` parameter. 

Do this now in the cell below. 

In [19]:
wv.most_similar(positive=('king', 'woman'), negative=['man'])

[('princess', 0.6069009304046631),
 ('revival', 0.594021737575531),
 ('queen', 0.5846858620643616),
 ('brunette', 0.5757460594177246),
 ('title', 0.5705699920654297),
 ('reminiscent', 0.5599520206451416),
 ('dancer', 0.5493637323379517),
 ('crown', 0.5428540110588074),
 ('coin', 0.5417349338531494),
 ('feminist', 0.5414919853210449)]

As we can see from the output above, our model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As we can see from the word in first place, 'reminiscent', our model is far from perfect. This is likely because we didn't give it too much training, or training data. However, for the small amount of training data it was given, the model still performs remarkably well! 

We'll see in the next lab that from a practical standpoint, one of the best things we can do for performance is to start by loading in the weights from an open-sourced model that has been trained for a very long time on a massive amount of data, such as the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless our text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, we learned how to train and use a Word2Vec model to created vectorized word embeddings!